In [7]:
import MySQLdb

In [8]:
db = MySQLdb.connect(host="localhost",  # host      # 127.0.0.1
                     user="root",       # username  # root
                     passwd="1234",     # password  # 1234
                     db="naver_movies")         # DB NAME   

db.set_character_set('utf8')

In [9]:
cur = db.cursor()

In [10]:
# IF(조건, 참값, 거짓값)
query = """
        CREATE TABLE IF NOT EXISTS naver_movie
            (code    INTEGER      NOT NULL      PRIMARY KEY,
             created_at    DATETIME DEFAULT CURRENT_TIMESTAMP,
             title   VARCHAR(100) NOT NULL,
             story   TEXT,
             open_date  DATE)
             ENGINE INNODB;
        """
cur.execute(query)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: Warning: (1050, "Table 'naver_movie' already exists")
  # This is added back by InteractiveShellApp.init_path()


0

In [11]:
query = """
        CREATE TABLE IF NOT EXISTS movie_comment(
            score    INTEGER    NOT NULL,
            content  TEXT       NOT NULL,
            movie_code INTEGER,
            writer   VARCHAR(100),
            write_date DATETIME,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            PRIMARY KEY (writer, write_date),
            FOREIGN KEY(movie_code)
            REFERENCES naver_movie(code) on delete cascade)
            ENGINE INNODB;
"""
cur.execute(query)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: Warning: (1050, "Table 'movie_comment' already exists")
  


0

In [12]:
import requests
from bs4 import BeautifulSoup
from urllib import parse

In [13]:
MOVIE_LIST_URL = 'https://movie.naver.com/movie/sdb/rank/rmovie.nhn'

# ?sel=pnt&date=20190616

In [14]:
def get_movie_list(start_page=1, end_page=1):
    URL = "https://movie.naver.com/movie/sdb/rank/rmovie.nhn"
    params = {
        'sel':'pnt',
        'date':20190616
    }
    
    movie_list = []
    for i in range(start_page, end_page+1):
        params['page'] = i
        
        resp = requests.get(URL, params)
        soup = BeautifulSoup(resp.content)
        
        list_ranking = soup.find(class_='list_ranking').find_all('tr')
        for movie_tr in list_ranking:
            try:
                code = movie_tr.find('a')['href'].split('=')[-1]
                title = movie_tr.find('a').text
            
                movie_list.append({
                    'code':code,
                    'title':title
                })
            except Exception:
                pass
            
    return movie_list

In [15]:
def get_movie_info(movie):
    code = movie.get('code')
    title = movie.get('title')
    
    base_url = 'https://movie.naver.com/movie/bi/mi/basic.nhn'
    params = {
        'code': code,
    }
    
    resp = requests.get(base_url, params)
    soup = BeautifulSoup(resp.content)
    
    open_date_tag = soup.\
    find('dl', class_='info_spec').\
    find('dd').find_all('span')[-1]
    
    # 여기부터
    open_date = ""
    for date in open_date_tag.find_all('a'):
        open_date += date.text
    
    open_date = "-".join(open_date.strip().\
                         split('.'))
    
    # 여기까지
    story = soup.find('div', class_='story_area')
    
    movie['story'] = story
    movie['open_date'] = open_date
    
    return movie

In [16]:
movie_list = get_movie_list()
movie_list[0]

{'code': '171539', 'title': '그린 북'}

In [17]:
get_movie_info(movie_list[0])

{'code': '171539', 'title': '그린 북', 'story': <div class="story_area">
 <div class="title_area">
 <h4 class="h_story"><strong class="blind">줄거리</strong></h4>
 </div>
 <h5 class="h_tx_story">언제 어디서든 바른 생활! 완벽한 천재 뮤지션 ‘돈 셜리’<br/>
 원칙보다 반칙! 다혈질 운전사 ‘토니’<br/>
 취향도, 성격도 완벽히 다른 두 남자의 특별한 우정이 시작된다!</h5>
 <p class="con_tx">1962년 미국, 입담과 주먹만 믿고 살아가던 토니 발레롱가(비고 모텐슨)는 교양과 우아함 그 자체인
 <br/> 천재 피아니스트 돈 셜리(마허샬라 알리) 박사의 운전기사 면접을 보게 된다.
 <br/> 
 <br/> 백악관에도 초청되는 등 미국 전역에서 콘서트 요청을 받으며 명성을 떨치고 있는 돈 셜리는
 <br/> 위험하기로 소문난 미국 남부 투어 공연을 떠나기로 결심하고,
 <br/> 투어 기간 동안 자신의 보디가드 겸 운전기사로 토니를 고용한다.
 <br/> 
 <br/> 거친 인생을 살아온 토니 발레롱가와 교양과 기품을 지키며 살아온 돈 셜리 박사.
 <br/> 생각, 행동, 말투, 취향까지 달라도 너무 다른 두 사람은
 <br/> 그들을 위한 여행안내서 ‘그린북’에 의존해 특별한 남부 투어를 시작하는데…</p>
 <button class="story_more" id="toggleMakingnoteButton" onclick="storyAndNote.toggleMakingnote();" type="button"><em class="blind">제작노트 보기</em></button><!-- N=a:mai.story -->
 </div>, 'open_date': '2019-01-09'}

In [18]:
# query = """
#         CREATE TABLE IF NOT EXISTS naver_movie
#             (code    INTEGER      NOT NULL      PRIMARY KEY,
#              created_at    DATETIME DEFAULT CURRENT_TIMESTAMP,
#              title   VARCHAR(100) NOT NULL,
#              story   TEXT,
#              open_date  DATE)
#         """
def insert_movie(movie):
    code = movie.get('code')
    title = movie.get('title')
    story = movie.get('story')
    
    story = str(story.text)
    open_date = movie.get('open_date')
    #story = db.escape_string(story)
    
    query = """
    INSERT INTO naver_movie
    (code, title, story, open_date)
    VALUES('{0}', '{1}', '{2}', '{3}');
    """.format(code, title, story, open_date)
    
    cur.execute(query)
    db.commit()

In [19]:
sample_movie = get_movie_info(movie_list[0])

In [20]:
insert_movie(sample_movie)

IntegrityError: (1062, "Duplicate entry '171539' for key 'PRIMARY'")

In [ ]:
from MySQLdb import IntegrityError

In [ ]:
movie_list = get_movie_list()
for movie in movie_list:
    movie = get_movie_info(movie)
    try:
        insert_movie(movie)
    except IntegrityError:
        pass
    except Exception:
        pass

In [21]:
for movie in movie_list:
    movie = get_movie_info(movie)
    code = movie.get('code')
    print(code)
    title = movie.get('title')
    print(title)
    story = movie.get('story')
    
    story = str(story.text)
    print(story)
    open_date = movie.get('open_date')
    print(open_date)

171539
그린 북


줄거리

언제 어디서든 바른 생활! 완벽한 천재 뮤지션 ‘돈 셜리’
원칙보다 반칙! 다혈질 운전사 ‘토니’
취향도, 성격도 완벽히 다른 두 남자의 특별한 우정이 시작된다!
 그들을 위한 여행안내서 ‘그린북’에 의존해 특별한 남부 투어를 시작하는데…박사.돈 셜리는 그 자체인
제작노트 보기

2019-01-09
174830
가버나움


줄거리

 -출생기록조차 없이 살아온 어쩌면 12살 소년 '자인'으로부터
제작노트 보기

2019-01-24
144906
베일리 어게인


줄거리

이번 생은 처음인 우리에게
견생 4회차, 베일리의 빅허그♥
 마침내 자신이 돌아온 진짜 이유를 깨닫고 어딘가로 달려가기 시작하는데…지!
제작노트 보기

2018-11-22
163788
알라딘


줄거리

 자스민 공주의 마음을 얻으려다 생각도 못했던 모험에 휘말리게 되는데…가
제작노트 보기

2019-05-23
169240
아일라


줄거리

포화 속 생에 다시 없을 운명 같은 만남, 그리고… 60년간의 그리움
한국전쟁 참전병 ‘슐레이만’과 5살 소녀 ‘아일라’의 가슴 뜨거운 감동 실화!
1950년 한국전쟁에 파병된 ‘슐레이만’은 칠흑 같은 어둠 속 홀로 남겨진 5살 소녀를 발견한다. 전쟁과 부모를 잃은 충격 속에 말을 잃은 소녀. ‘슐레이만’은 소녀에게 터키어로 ‘달’이라는 뜻의 ‘아일라’라는 이름을 지어주고 함께 부대로 향한다. 서로에게 무엇과도 바꿀 수 없는 가장 소중한 존재가 된 두 사람. 그러나 행복도 잠시, ‘슐레이만’은 종전과 함께 고국으로 돌아가라는 명령을 받게 되고, ‘아일라’를 끝까지 지켜내기 위해 모두의 반대 속에 자신의 선택을 감행하게 되는데…  전 세계를 감동시킨 위대한 이야기가 찾아온다!
제작노트 보기

2018-06-21
151196
원더


줄거리

“넌 못생기지 않았어, 네게 관심있는 사람은 알게 될 거야”
헬멧 속에 숨었던 아이 ‘어기', 세상 밖으로 나오다!
누구보다 위트 있고 호기심 많은 매력 부자 ‘어기'(제이콥 트렘블레이)

35901
살인의 추억


줄거리

연쇄살인 실화극 | 1986년 시골마을, 두 형사에겐 모든 것이 처음이었다. | 당신은 지금 어디에 있는가 | 미치도록 잡고 싶었다. 당신은 누구인가 | 미치도록 잡고 싶었습니다. 당신은 누구십니까
1986년 경기도. 젊은 여인이 무참히 강간, 살해당한 시체로 발견된다. 2개월 후, 비슷한 수법의 강간살인사건이 연이어 발생하면서 사건은 세간의 주목을 받기 시작하고, 일대는 연쇄살인이라는 생소한 범죄의 공포에 휩싸인다.  사건 발생지역에 특별수사본부가 설치되고, 수사본부는 구희봉 반장(변희봉 분)을 필두로 지역토박이 형사 박두만(송강호 분)과 조용구(김뢰하 분), 그리고 서울 시경에서 자원해 온 서태윤(김상경 분)이 배치된다. 육감으로 대표되는 박두만은 동네 양아치들을 족치며 자백을 강요하고, 서태윤은 사건 서류를 꼼꼼히 검토하며 사건의 실마리를 찾아가지만, 스타일이 다른 두 사람은 처음부터 팽팽한 신경전을 벌인다. 용의자가 검거되고 사건의 끝이 보일 듯 하더니, 매스컴이 몰려든 현장 검증에서 용의자가 범행 사실을 부인하면서 현장은 아수라장이 되고, 구반장은 파면 당한다.  수사진이 아연실색할 정도로 범인은 자신의 흔적을 남기지 않는다. 살해하거나 결박할 때도 모두 피해자가 착용했거나 사용하는 물품을 이용한다. 심지어 강간사 일 경우, 대부분 피살자의 몸에 떨어져 있기 마련인 범인의 음모 조차 단 하나도 발견되지 않는다. 후임으로 신동철 반장(송재호 분)이 부임하면서 수사는 활기를 띠기 시작한다. 박두만은 현장에 털 한 오라기 남기지 않는다는 점에 착안, 근처의 절과 목욕탕을 뒤지며 무모증인 사람을 찾아 나서고, 사건 파일을 검토하던 서태윤은 비오는 날, 빨간 옷을 입은 여자가 범행대상이라는 공통점을 밝혀낸다.  선제공격에 나선 형사들은 비오는 밤, 여경에게 빨간 옷을 입히고 함정 수사를 벌인다. 그러나 다음날 아침 돌아오는 것은 또다른 여인의 끔찍한 사체. 사건은 해결의 실마리를 다시 감추고 냄비처럼 들끊는 언론은 일선 형사

18543
서유기 2 - 선리기연


줄거리

&lt서유기 월광보합&gt의 완결편. 지존보는 500년전의 세계로 와서 자하란 여자를 만나는데 이 여자는 500년 후에 백정정과 춘삼십낭의 사부가 되는 반사대사이다. 자하는 바로 여래신등의 심지로 언니와 한몸이나 자신의 몽중인을 찾으러 속세로 나온다. 자신의 자청보검을 뽑을 수 있는 사람이 그녀의 꿈속의 낭군. 지존보는 자하가 월광보합을 갖고 있어서 어떻게든 월광보합으로 500년후의 세상으로 가서 백정정을 구하려고 한다. 한편 자신의 배필감은 자신의 자청보검을 뽑을 수 있다고 믿고 있던 자하는 지존보가 자청보검을 뽑는 것을 보고 자신의 낭군으로 점찍어 놓는다. 한편 지존보는 자하에게서 월광보합을 얻으려고 사랑한다고 거짓말을 한다. 그러나 지존보에게 사랑하는 여자가 있다는 걸 안 자하는 떠나고 다시 사부님을 만난 지존보는 사제들과 함께 우마왕에게 잡힌다. 우마왕은 지존보를 자신의 여동생인 향향과 혼인시키려 하고 자신이 생명을 구해줬던 자하를 자신의 첩으로 만드려고 한다. 몰래 도망을 친 지존보는 거미줄동굴로 가 백정정을 만난다. 그는 500년후의 세상으로 돌아갈 필요가 없게되자 당장 그녀와 혼인을 하려한다. 하지만 그의 마음은 이미 자하에게로 기울어져 있었으니. 이런 지존보의 마음을 안 백정정은 그의 곁을 떠나고 지존보는 관세음보살을 찾아가 손오공이 되겠다고 말하며 절대로 여자를 사랑하지 않겠다고 맹세한다. 그리고 저팔계와 사오정과 함께 당삼장을 구하러 우마왕의 성에 가서 결전을 벌이고 자하도 구해낸다. 지존보의 사랑을 확인하는 자하. 그녀는 일 대 혈전속에서 위기에 처한 손오공을 구하다가 우마왕의 칼에 맞아죽고 우마왕은 손오공의 손에 죽는다. 이때 성이 무너지려하자 손오공 일행은 월광보합으로 500년후의 세상으로 온다. 이때의 세상은 요괴가 사라지고 태평스러운 세상이다. 손오공 일행은 길을 떠나면서 지존보와 자하가 다시 만나서 서로 사랑을 확인하는 장면을 보게 된다

2010-06-01
19079
굿 윌 헌팅




In [22]:
def get_comment_list(code, last_page=1):
    url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn'
    params = {
        'code': code,
        'type': 'after',
        'order': 'newest'
    }
    result_list = []
    for page in range(1, last_page+1):
        params['page'] = page
        
        resp = requests.get(url, params)
        soup = BeautifulSoup(resp.content)
        reple_list = soup.find('div', class_='score_result').find_all('li')
        
        for reple in reple_list:
            score = reple.find(class_='star_score').text.strip()
            content = reple.find('p')
            name_n_date = content.find_next_sibling('dl').find_all('em')
            
            content = content.text
            
            nickname = name_n_date[0]
            date = name_n_date[1]
            result_list.append({
                'write_name':nickname,
                'write_date': date.text,
                'content': content,
                'score': score,
                'code':code
            })
    return result_list

In [23]:
comment=get_comment_list(18847)

In [24]:
comment[0]['write_date']

'2019.06.17 15:25'

In [36]:
def insert_comment(comment):
    score = comment.get('score')
    writer = comment.get('write_name').text
    write_date = comment.get('write_date')
    write_date = '-'.join(write_date.split('.'))
    
    content = comment.get('content')
    movie_code = comment.get('code')
    
    query = """
    INSERT INTO movie_comment
    (score, writer, write_date, content, movie_code)
    VALUES('{}', '{}', '{}', '{}', '{}')
    """.format(score, writer, write_date, content, movie_code)
    
    cur.execute(query)
    db.commit()

In [37]:
import datetime

In [38]:
for movie in movie_list:
    try:
        comment_list = get_comment_list(movie['code'])
        for comment in comment_list:
            print(comment)
            insert_comment(comment)
    except Exception:
        pass

{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15816086, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>봉봉(lemo****)</span>
</a>
</em>, 'write_date': '2019.06.17 00:07', 'content': '따뜻한 브로맨스 로드무비.  ', 'score': '10', 'code': '171539'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15812565, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>Windy(appl****)</span>
</a>
</em>, 'write_date': '2019.06.15 21:46', 'content': '평점 클라스 봐라 ㅋㅋ 너무한거 아니냐?  ', 'score': '1', 'code': '171539'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15811976, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>lunasea(luna****)</span>
</a>
</em>, 'write_date': '2019.06.15 17:51', 'content': '블루레이로 감삼.두 배우의 연기가 대단하고 내용 전개 중에 심한 갈등이 나올거 같았는데 나오지 않아 편하고 재밌게 볼수 있었다.  ', 'score': '10', 'code': '171539'}
{'write_name': <

{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15804227, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>해솔이(luom****)</span>
</a>
</em>, 'write_date': '2019.06.13 03:16', 'content': '뭉클함과 감동..오래오래 형제의나라로 기억하고싶었다  ', 'score': '10', 'code': '169240'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15779031, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>puma****</span>
</a>
</em>, 'write_date': '2019.06.06 14:37', 'content': '네이버 평점 한줄분석에 이영화를 가장 좋아하는 그룹은 30대 남자입니다..라는걸 보고 여기 평점에 눈물흘리고 봤다던 사람들은 대부분 나랑 같은 30대 남자구나..나만운게 아니라 다행이여 ㅋㅋㅋㅋ  ', 'score': '10', 'code': '169240'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15771698, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>최동용(choi****)</span>
</a>
</em>, 'write_date': '2019.06.05 01:58', 'content': '진짜 잘안우는데 눈물이 남  ', 'score': '10', 

{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15814430, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>fres****</span>
</a>
</em>, 'write_date': '2019.06.16 15:02', 'content': '10점이상을 못준다는게 이 영화에게 미안할 뿐  ', 'score': '10', 'code': '17421'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15814616, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>허성희(hj89****)</span>
</a>
</em>, 'write_date': '2019.06.16 16:00', 'content': '배우님들 연기 너무 잘하시고 많은 생각을 느끼게 해주는 그런 영화!! 슬프고 감동적이예요  ', 'score': '10', 'code': '154667'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15813818, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>ENOO(qudj****)</span>
</a>
</em>, 'write_date': '2019.06.16 10:01', 'content': '뻔한 내용이라고는 하지만 그래도 그안에서의 짜임새 있는 연출력과 편집은 쉬운게 아니잖아요. 그런면에서 좋았구 무어보다 아역배우님의 연기가 감탄스러웠습니다. 잘봤어요!  ',

{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15810170, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>blacksnow(blac****)</span>
</a>
</em>, 'write_date': '2019.06.15 00:12', 'content': '어지간해서 잘 안다는데  ', 'score': '10', 'code': '18988'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15805517, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>리아(kanz****)</span>
</a>
</em>, 'write_date': '2019.06.13 16:09', 'content': '아주 무난하게 만들어졌다 생각했는데 러닝타임 170분... 이 정도 길이를 무난하게 만든 거면 탄탄하게 만든 거다.  ', 'score': '7', 'code': '18988'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15804217, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>그뤠잇터(pssh****)</span>
</a>
</em>, 'write_date': '2019.06.13 03:07', 'content': '요새 나오는 히어로 영화보다 이게 훨씬 낫다  ', 'score': '10', 'code': '18988'}
{'write_name': <em>
<a hr

{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15754772, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>박유(uup9****)</span>
</a>
</em>, 'write_date': '2019.06.01 20:45', 'content': '교훈이있는 영화. 감사합니다  ', 'score': '10', 'code': '82432'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15816555, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>리겔(saro****)</span>
</a>
</em>, 'write_date': '2019.06.17 03:40', 'content': '와 진짜 너무넘. 긴장감 넘치고 재밌었다.크림슨타이드보다 2배는 재밌다.  ', 'score': '10', 'code': '92125'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15815940, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>토비아스(dlgh****)</span>
</a>
</em>, 'write_date': '2019.06.16 23:30', 'content': '기대하지 않고 봤는데 생각보다 재밌게 보았다  ', 'score': '8', 'code': '92125'}
{'write_name': <em>
<a href="#" onclick="javascript:s

{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15816690, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>정132487(lkjh****)</span>
</a>
</em>, 'write_date': '2019.06.17 07:41', 'content': '명작..  너무 재밋다 2편은 안나오겠지?ㅜ  ', 'score': '10', 'code': '32686'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15809601, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>fly2****</span>
</a>
</em>, 'write_date': '2019.06.14 20:27', 'content': '극장에서 본 당신이 승자. 음악과 어루어지는 연출과 소름돋는 음악의 연속.  ', 'score': '10', 'code': '106360'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15806647, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>맛난감자(rd72****)</span>
</a>
</em>, 'write_date': '2019.06.13 23:08', 'content': '다양성에 매몰된, 착함 콤플렉스  ', 'score': '1', 'code': '106360'}
{'write_name': <em>
<a href="#" onclick="javasc

</em>, 'write_date': '2019.06.09 14:01', 'content': '이 영화 감동적이에요윌슨을 친구처럼 대하는 아저씨~~^^ 꼭 보세요추천드려요~~^^????  ', 'score': '10', 'code': '31162'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15786729, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>미래(basi****)</span>
</a>
</em>, 'write_date': '2019.06.07 23:39', 'content': '인생영화로 꼭  추천합니다  ', 'score': '10', 'code': '31162'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15786543, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>dltkdgks50(doff****)</span>
</a>
</em>, 'write_date': '2019.06.07 23:01', 'content': 'You have to go home과 마지막 택배 놔두며 적은 메모를 보며 정말  ', 'score': '10', 'code': '31162'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15764207, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>Cloudjumper(aloh****)</span>
</a>
</em>, '

{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15786734, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>SEOULFORESTG(nece****)</span>
</a>
</em>, 'write_date': '2019.06.07 23:41', 'content': '정말 저의 인생영화입니다  ', 'score': '10', 'code': '12482'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15748658, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>날적이(gour****)</span>
</a>
</em>, 'write_date': '2019.05.31 22:47', 'content': '탱고 장면만 생각나서 다시 봤는데... 마지막 장면 대박!!! 전율을 느끼게하는 명연설이 감동을 준다. Snich. "This is not 판문점" 맨해튼 월도프 아스코리아 호텔. 경찰 속도위반 단속 지점인 덤보 거리가 눈에 들어온다...  ', 'score': '10', 'code': '12482'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15730625, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>윤뉴(dbsd****)</span>
</a>
</em>, 'write_date': '2019.05.28 23:12', 'content': '내 인생영화 정말 명작임

{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15817121, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>Thesis(fine****)</span>
</a>
</em>, 'write_date': '2019.06.17 12:46', 'content': '이영화를 보고 암이 나았습니다  ', 'score': '10', 'code': '37886'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15817022, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>Han(ehrn****)</span>
</a>
</em>, 'write_date': '2019.06.17 11:45', 'content': '이 영화를 보고 발기부전과 조루가 나았습니다... 신이시여!  ', 'score': '10', 'code': '37886'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15816581, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>RevengEmUST(mkki****)</span>
</a>
</em>, 'write_date': '2019.06.17 04:03', 'content': "재밌게 신앙고백하듯 놀고 있는데 진지 빨고 별점1개주고 '정상인 분들을 위해 씁니다. 절대 보지마세요ㅋㅋ'이 OO하는 새끼들 귀싸대기 오른쪽아래서부터 왼쪽위방향으로 쎄개 올려치고 싶다  ", 

{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15816722, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>just****</span>
</a>
</em>, 'write_date': '2019.06.17 08:23', 'content': '생에 저런사람을 만날수있을까? 저런 인생의 선생님을 만날수있을까? 없다면 내가 누군가의 저런사람이되고싶다는 생각을하게끔 만들어주는 영화  ', 'score': '10', 'code': '19079'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15796144, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>해례본학원(ssen****)</span>
</a>
</em>, 'write_date': '2019.06.10 03:52', 'content': '우린 가족에서 다시 부부였다 ...  ', 'score': '10', 'code': '73476'}
{'write_name': <em>
<a href="#" onclick="javascript:showPointListByNid(15728493, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>안유진(kmo7****)</span>
</a>
</em>, 'write_date': '2019.05.28 00:22', 'content': 'kioluikjhfg  ', 'score': '1', 'code': '73476'}
{'write_name': <em>
<a href="

In [31]:
comment_li = get_comment_list(174830)

In [35]:
comment_li[0]['write_name'].text

'\n\nWindy(appl****)\n\n'